In [1]:
import turicreate
song_data = turicreate.SFrame('song_data.sframe/')

In [2]:
song_data

user_id,song_id,listen_count,title,artist
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOAKIMP12A8C130995,1,The Cove,Jack Johnson
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Paco De Lucia
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBXHDL12A81C204C0,1,Stronger,Kanye West
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBYHAJ12A6701BF1D,1,Constellations,Jack Johnson
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODACBL12A8C13C273,1,Learn To Fly,Foo Fighters
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODDNQT12A6D4F5F7E,5,Apuesta Por El Rock 'N'Roll ...,Héroes del Silencio
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODXRTY12AB0180F3B,1,Paper Gangsta,Lady GaGa
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFGUAY12AB017B0A8,1,Stacked Actors,Foo Fighters
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFRQTD12A81C233C0,1,Sehr kosmisch,Harmonia
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOHQWYZ12A6D4FA701,1,Heaven's gonna burn youreyes ...,Thievery Corporationfeat. Emiliana Torrini ...


## count the number of unique listeners of each artist

In [23]:
artists = ['Kanye West', 'Foo Fighters', 'Taylor Swift', 'Lady GaGa']
for s in artists:
    df = song_data[song_data['artist'] == s]
    users = df['user_id'].unique()
    print (s, len(users))

Kanye West 2522
Foo Fighters 2055
Taylor Swift 3246
Lady GaGa 2928


## calculate the total listen count of each artist \ find out the most and least popular artist

In [32]:
listen = song_data.groupby(key_column_names='artist', operations={'total_count': turicreate.aggregate.SUM('listen_count')})

In [33]:
listen.sort(key_column_names = 'total_count', ascending = False)[0]

{'artist': 'Kings Of Leon', 'total_count': 43218}

In [34]:
listen.sort(key_column_names = 'total_count', ascending = False)[-1]

{'artist': 'William Tabbert', 'total_count': 14}

## Train an item_similarity_recommender

In [35]:
train_data,test_data = song_data.random_split(.8,seed=0)

In [36]:
personalized_model = turicreate.item_similarity_recommender.create(train_data,
                                                                  user_id = 'user_id',
                                                                  item_id = 'song')

Warning: Ignoring columns song_id, listen_count, title, artist;

To use one of these as a rating column, specify the column name to be used as target

and use a method that allows the use of a target.

Preparing data set.

Data has 893580 observations with 66085 users and 9952 items.

...

| 1.73s                               | 100              | 9952            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 2.83126s

In [37]:
subset_test_users = test_data['user_id'].unique()[0:10000]

In [40]:
recommend = personalized_model.recommend(subset_test_users,k=1)
recommend

recommendations finished on 1000/10000 queries. users per second: 9869.33

recommendations finished on 2000/10000 queries. users per second: 12267.7

recommendations finished on 3000/10000 queries. users per second: 13585.7

recommendations finished on 4000/10000 queries. users per second: 14366.4

recommendations finished on 5000/10000 queries. users per second: 14749.2

...

recommendations finished on 7000/10000 queries. users per second: 13822.9

recommendations finished on 8000/10000 queries. users per second: 14184.7

recommendations finished on 9000/10000 queries. users per second: 14462.3

recommendations finished on 10000/10000 queries. users per second: 14285

user_id,song,score,rank
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Cuando Pase El Temblor -Soda Stereo ...,0.019450453611520622,1
c067c22072a17d33310d7223d7b79f819e48cf42 ...,Grind With Me (ExplicitVersion) - Pretty Ricky ...,0.04594243764877319,1
f6c596a519698c97f1591ad89f540d76f6a04f1a ...,Hey_ Soul Sister - Train,0.023892953991889954,1
696787172dd3f5169dc94deef97e427cee86147d ...,Senza Una Donna (WithoutA Woman) - Zucchero / ...,0.01702657767704555,1
3a7111f4cdf3c5a85fd4053e3cc2333562e1e0cb ...,Heartbreak Warfare - JohnMayer ...,0.029841651519139607,1
532e98155cbfd1e1a474a28ed96e59e50f7c5baf ...,Jive Talkin' (AlbumVersion) - Bee Gees ...,0.011828865323747908,1
ee43b175ed753b2e2bce806c903d4661ad351a91 ...,Ricordati Di Noi -Valerio Scanu ...,0.03051712115605672,1
e372c27f6cb071518ae500589ae02c126954c148 ...,Fall Out - The Police,0.08196720480918884,1
83b1428917b47a6b130ed471b09033820be78a8c ...,Clocks - Coldplay,0.04285883903503418,1
39487deef9345b1e22881245cabf4e7c53b6cf6e ...,Black Mirror - ArcadeFire ...,0.04177376857170692,1


In [45]:
recommend_pop = recommend.groupby(key_column_names='song', operations={'total_count': turicreate.aggregate.COUNT()})
recommend_pop.sort(key_column_names = 'total_count', ascending = False)

song,total_count
Undo - Björk,435
Secrets - OneRepublic,384
Revelry - Kings Of Leon,225
You're The One - DwightYoakam ...,164
Fireflies - CharttraxxKaraoke ...,120
Sehr kosmisch - Harmonia,96
Horn Concerto No. 4 in Eflat K495: II. Romance ...,96
Hey_ Soul Sister - Train,91
OMG - Usher featuringwill.i.am ...,63
Dog Days Are Over (RadioEdit) - Florence + The ...,46
